In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings 

warnings.filterwarnings("ignore")

In [ ]:
path = "../../../Data/ai_human_images/"
train = pd.read_csv(path+"train.csv", index_col=0)
test = pd.read_csv(path+"test.csv")

In [ ]:
import os

# Create a new column with full paths
train["file_name"] = train["file_name"].apply(lambda x: os.path.join(path, x))
test["id"] = test["id"].apply(lambda x: os.path.join(path, x))

# Rename the image columns
train = train.rename(columns = {"file_name": "images"})
test = test.rename(columns = {"id": "images"})

In [ ]:
test.head()

In [ ]:
train.head()